| Nome e cognome      | Insegnamento | Anno Accademico    |
| :---        |    :----   |          ---: |
| Alessandra Sirghe      | Prova finale       | 2024/2025   |

#  “Analisi computazionale diacronica dell’evoluzione semantica di parole legate all’esperienza femminile nell’area anglofona (XVIII–XX secolo) tramite Word Embeddings” (Notebook 2)



## Presentazione

Questo notebook sarà dedicato all'allenamento di due modelli di [*representation learning*](https://en.wikipedia.org/wiki/Feature_learning) per la creazione automatica di *Word Embeddings*, sfruttando le potenzialità del metodo a *subwords* della libreria [fastText](https://fasttext.cc/docs/en/python-module.html) sviluppata da Facebook. Tali modelli saranno allenati con i corpora creati secondo il procedimento descritto nel primo notebook.

Gli *Word Embeddings* sono essenziali nella *Vector Semantics*, ossia la forma standard di rappresentazione del significato delle parole nell'ambito dell'[NLP](https://en.wikipedia.org/wiki/Natural_language_processing). Un *embedding* è, nello specifico, la rappresentazione di una parola sotto forma di un vettore. Il suo uso si basa sull'ipotesi distribuzionale di Joos, Harris e Firth (1950; 1954; 1957), secondo cui il significato di una parola è definito dalle parole che popolano il contesto a essa circostante: due parole saranno tanto più simili quanto simili sono le parole vicine che ne definiscono il contesto (Jurafsky e Martin 2025, 4-5). Nella forma elementare di creazione dei vettori si calcola quante volte ciascuna parola contenuta in un testo appare in una stessa finestra di contesto con le altre parole in esso contenute, fino ad ottenere delle matrici di co-occorrenza dove non si esclude la presenza di zeri. Pertanto, le dimensioni di una parola-vettore (la "lunghezza", o da quanti numeri sono costituiti) sono generalmente definite, a questo livello base, dal numero di parole contenute nell'intero "vocabolario" rappresentato da tutte le parole presenti nel testo o corpus, il che le può rendere particolarmente grandi e sparse. Gli *embeddings*, come quelli generati da fastText, si differenziano in questo, essendo rappresentazioni più dense rispetto a quelle dei vettori tradizionali e raggiungendo in genere un numero di dimensioni non superiori alle mille. Stando a Jurafsky e Martin, questa diminuzione del numero di dimensioni:

>  . . . requires our classifiers to learn far fewer weights than if we represented words as 50,000-dimensional vectors, and the smaller parameter space possibly helps with generalization and avoiding overfitting (Jurafsky e Martin 2025, 10).

Gli algoritmi basati su metodi come quelli di fastText e [word2vec](https://radimrehurek.com/gensim/models/word2vec.html), difatti, non danno come risultati delle matrici di co-occorrenza, bensì si basano su un compito di predizione binaria (sì/no) sulla possibilità di una parola di apparire accanto a un'altra: così il modello può creare gli *embeddings* in modalità di *self-supervised learning* usando come risposta corretta (*gold standard*) la parola che effettivamente appare subito dopo (Jurafsky e Martin 2025, 11). In aggiunta, fastText permette di ovviare al problema di ricerca di affinità semantiche di parole che non erano presenti nel corpus di *testing* del modello: ciascuna parola può essere rappresentata come la somma degli *embeddings* delle cosiddette *subwords* che le rappresentano (nel caso della parola *where*, quindi, come la somma degli *embeddings* *<wh, whe, her, ere, re>*: v. Jurafsky e Martin 2025, 16).   

Il valore di "somiglianza" tra due parole-vettore, infine, è dato dalla [*cosine similarity*](https://en.wikipedia.org/wiki/Cosine_similarity), ovvero dal calcolo del coseno dell'angolo che si forma tra due vettori e che può essere rappresentato da un numero compreso tra 0 e 1: più si avvicina a quest'ultimo e più le due parole rappresentate da quei vettori saranno da considerarsi simili (Jurafsky e Martin 2025, 8-10).

In ordine di presentazione, questi sono i contenuti del notebook:

1. Installazione e import dei moduli e delle librerie.
2. Analisi delle affinità rilevate per il primo periodo (1700-1832).
3. Analisi delle affinità rilevate per il secondo periodo (1832-1932).
4. Analisi e riepilogo di alcuni cambi semantici eventualmente rilevati.
6. Breve descrizione di studi in cui i *Word Embeddings* (d'ora in poi abbreviati in *WE*) vengono usati in ambito DH.

Ancora prima delle parti di analisi verranno effettuate delle prove iniziali con campioni di parole non necessariamente associate al tema della stessa e, solo successivamente, si proseguirà seguendo un percorso concettuale già inerente al suo scopo specifico; quando verrà ritenuto utile si offrirà anche una visualizzazione dei risultati, ottenuta grazie alle librerie **scikit-learn**, **plotly** e **numpy**.

Un'ultima e breve specifica tecnica riguarda l'uso, per la creazione e il funzionamento del codice di questo notebook, di Visual Studio Code a causa di una maggiore lentezza nella sua esecuzione nell'ambiente di Google Colab. Eventuali installazioni e import di librerie che si potrebbero considerare come aggiuntive o non strettamente necessarie nell'ambiente di Colab sono quindi da considerarsi come parte di questo cambiamento. La versione di Python utilizzata nel *virtual environment* di VSC è la 3.12.9.  



## Installazione e import dei moduli/librerie necessari

In [3]:
#%pip install fasttext-wheel
import fasttext #dopo l'installazione, in VSC è sufficiente importarlo negli usi successivi del notebook

In [ ]:
#%pip install scikit-learn

  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.7 MB 4.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.3/8.7 MB 3.9 MB/s eta 0:00:02
   --------- ------------------------------ 2.1/8.7 MB 3.8 MB/s eta 0:00:02
   --------------- ------------------------ 3.4/8.7 MB 4.3 MB/s eta 0:00:02
   -------------------- ------------------- 4.5/8.7 MB 4.5 MB/s eta 0:00:01
   ------------------------- -------------- 5.5/8.7 MB 4.6 MB/s eta 0:00:01
   ------------------------------ --------- 6.6/8.7 MB 4.7 MB/s eta 0:00:01
   ------------------------------------ --- 7.9/8.7 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 4.9 MB/s eta 0:00:00
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
   ---------------------------------------- 0.0/38.5 MB ? eta -


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#%pip install --upgrade nbformat
#aggiunto causa errore dato durante la creazione del plot

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#%pip install plotly

  Using cached plotly-6.2.0-py3-none-any.whl.metadata (8.5 kB)
Using cached plotly-6.2.0-py3-none-any.whl (9.6 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from sklearn.decomposition import PCA
import plotly.express as px
import numpy as np
from numpy import dot
from numpy.linalg import norm

In [ ]:
import plotly.graph_objects as go

In [ ]:
#%pip install pandas
#installazione necessaria per errore nel momento della creazione del plot
import pandas as pd

## Creazione dei modelli e prime osservazioni

La creazione dei modelli non ha previsto modifiche ai parametri standard già presenti nel metodo *train_unsupervised* di fastText: le dimensioni dei *WE* sono state lasciate a 100 e la dimensione della finestra di contesto da prendere in considerazione per le affinità semantiche è rimasta a ± 5. Quest'ultimo parametro viene ripreso in molte documentazioni di progetti analoghi a questo e si basa sulle osservazioni di Levy e Goldberg (2014), riprese anche da Jurafsky e Martin tra le proprietà semantiche degli *embeddings*: una finestra di contesto così ampia permette di tralasciare le affinità funzionali e sintattiche delle parole per catturare quelle più propriamente semantiche o di appartenenza allo stesso tema/argomento (2025, 17). Il modello specifico utilizzato come standard è lo *skipgram*, ovvero quello adatto a calcolare la probabilità che una parola appaia nello stesso contesto di un'altra parola *target*.

L'input per entrambi è stato un file TXT in cui sono stati scritti i *tokens*, separati da uno spazio (" "), dei testi ripuliti nel notebook precedente e nominati secondo il loro periodo di appartenenza.   

In [ ]:
first_period_model = fasttext.train_unsupervised("1700-1831.txt")

In [ ]:
first_period_model.save_model("first_period_model.bin")

In [ ]:
second_period_model = fasttext.train_unsupervised("1832-1932.txt")

In [ ]:
second_period_model.save_model("second_period_model.bin")

Per fare un rapido testing si è pensato di creare una lista di parole da poter usare per entrambi i modelli, senza che, come già detto in precedenza, i termini fossero necessariamente correlati all'argomento della successiva analisi.

```
trial_words = ["dog", "house", "wool", "woman", "freedom", "novel", "revolution", "unique", "stubborn", "child"]

```

Questa è stata passata successivamente come argomento di una funzione molto semplice, che permette di ottenere e visualizzare quelle parole che, con maggior probabilità, appaiono nello stesso contesto della parola di turno della lista. Tali parole verranno identificate nel resto del progetto come "vicini" o con l'espressione "semanticamente affini", il che ovviamente non indica necessariamente relazioni semantiche di sinonimia, bensì anche di "somiglianza" (*similarity*: la stessa relazione esistente tra due parole come "caffè" e "tazza"), "associazione" (*association* o *relatedness*), iperonimia o meronimia e antonimia (Jurafsky e Martin 2025, 2–4).

In [ ]:
def testing_the_model(list_of_trial_words, model):
  if list_of_trial_words:

    for word in list_of_trial_words:
        neighbours = model.get_nearest_neighbors(word)
        print(f"Nearest neighbours for '{word}':")
        for neighbour in neighbours:
            print(f"  {neighbour[1]} (similarity: {neighbour[0]:.4f})")
        print("\n")

  else:
    print("No list of words found.")

    return neighbours

In [ ]:
trial_words = ["dog", "house", "wool", "woman", "freedom", "novel", "revolution", "unique", "stubborn", "child"]

In [ ]:
first_period_model = fasttext.load_model("first_period_model.bin")

In [ ]:
testing_the_model(trial_words, first_period_model)

Nearest neighbours for 'dog':
  cat (similarity: 0.8215)
  dogs (similarity: 0.8185)
  rat (similarity: 0.7983)
  bitch (similarity: 0.7927)
  cats (similarity: 0.7618)
  squirrel (similarity: 0.7592)
  spuck (similarity: 0.7560)
  jackall (similarity: 0.7557)
  weasel (similarity: 0.7541)
  hare (similarity: 0.7426)


Nearest neighbours for 'house':
  meetinghouse (similarity: 0.8034)
  courthouse (similarity: 0.7946)
  southouse (similarity: 0.7857)
  houses (similarity: 0.7795)
  hall (similarity: 0.7767)
  cowhouse (similarity: 0.7670)
  chambers (similarity: 0.7663)
  outhouse (similarity: 0.7640)
  stowhill (similarity: 0.7561)
  scothouse (similarity: 0.7547)


Nearest neighbours for 'wool':
  woollen (similarity: 0.8339)
  wools (similarity: 0.7996)
  woolen (similarity: 0.7969)
  woollens (similarity: 0.7770)
  woolf (similarity: 0.7655)
  woold (similarity: 0.7579)
  broadcloths (similarity: 0.7565)
  hemp (similarity: 0.7553)
  cotton (similarity: 0.7506)
  manufactured (sim

A mio parere, i risultati apparsi, pur essendo influenzati dall'assenza della lemmatizzazione nella fase di *preprocessing* dei testi, contengono dei termini sufficientemente interessanti e passabili di approfondimento. Infatti, pur essendo i punteggi (calcolati sulla [*cosine similarity*](https://en.wikipedia.org/wiki/Cosine_similarity)) attribuiti a ciascun "vicino" più elevati per le forme flesse, allargando la visuale e permettendo a più risultati di apparire, si possono incontrare termini diversi e con valori di somiglianza pur sempre abbastanza elevati.   

In [ ]:
first_period_model.get_nearest_neighbors("revolution", k=20)

[(0.8540246486663818, 'revolutionized'),
 (0.8351438641548157, 'revolutions'),
 (0.8349030613899231, 'revolutionizing'),
 (0.8141692876815796, 'revolutionize'),
 (0.7976304292678833, 'revolutionary'),
 (0.7879579067230225, 'revolutioners'),
 (0.7733152508735657, 'revolutionist'),
 (0.7702443599700928, 'revolutionists'),
 (0.7355585694313049, 'devolution'),
 (0.6985613107681274, 'downfall'),
 (0.6908667087554932, 'restoration'),
 (0.682282030582428, 'accession'),
 (0.6683515310287476, 'revolute'),
 (0.6681520938873291, 'reformation'),
 (0.6664270758628845, 'commencement'),
 (0.663044810295105, 'commencements'),
 (0.6581826210021973, 'commotions'),
 (0.6574357151985168, 'insurrection'),
 (0.6565965414047241, 'preformation'),
 (0.6534642577171326, 'downfal')]

La stessa piccola prova è stata ovviamente effettuata anche per il secondo modello, riscontrando dei risultati dalla qualità analoga:

In [ ]:
second_period_model = fasttext.load_model("second_period_model.bin")

In [ ]:
testing_the_model(trial_words, second_period_model)

Nearest neighbours for 'dog':
  dogs (similarity: 0.8521)
  sheepdog (similarity: 0.7904)
  kennel (similarity: 0.7885)
  terrier (similarity: 0.7761)
  cat (similarity: 0.7659)
  spaniel (similarity: 0.7640)
  puppy (similarity: 0.7617)
  puppies (similarity: 0.7588)
  mastiff (similarity: 0.7574)
  cur (similarity: 0.7521)


Nearest neighbours for 'house':
  cottage (similarity: 0.8334)
  mansion (similarity: 0.8253)
  hall (similarity: 0.8144)
  place (similarity: 0.8099)
  room (similarity: 0.7971)
  lodge (similarity: 0.7865)
  chambery (similarity: 0.7801)
  houses (similarity: 0.7795)
  lodgings (similarity: 0.7791)
  bathhouse (similarity: 0.7786)


Nearest neighbours for 'wool':
  woollen (similarity: 0.8217)
  wools (similarity: 0.8111)
  wooll (similarity: 0.8072)
  cotton (similarity: 0.8046)
  woolen (similarity: 0.7885)
  woollens (similarity: 0.7671)
  woolens (similarity: 0.7548)
  cottons (similarity: 0.7499)
  cottony (similarity: 0.7483)
  silk (similarity: 0.7470)



In [ ]:
second_period_model.get_nearest_neighbors("revolution", k=20)

[(0.9188588857650757, 'revolutions'),
 (0.8800637125968933, 'revolutionary'),
 (0.8597140908241272, 'revolutionaries'),
 (0.8569396734237671, 'revolutionised'),
 (0.8368433713912964, 'revolutionizing'),
 (0.8359460830688477, 'revolutionized'),
 (0.8346330523490906, 'revolutionise'),
 (0.7893284559249878, 'revolutionists'),
 (0.7888335585594177, 'devolution'),
 (0.788006603717804, 'revolutionist'),
 (0.7848259806632996, 'revolutionize'),
 (0.7510699033737183, 'anarchy'),
 (0.7339141964912415, 'insurrections'),
 (0.726184606552124, 'evolutionary'),
 (0.7237547636032104, 'insurrection'),
 (0.7237227559089661, 'militarization'),
 (0.7221773862838745, 'insurrectionary'),
 (0.7220712304115295, 'nationalization'),
 (0.7192673683166504, 'revolute'),
 (0.7190159559249878, 'republic')]

Per visualizzare i dati in maniera più intuitiva e avere un'ulteriore prova del buon funzionamento del modello, si possono creare dei plot.

La funzione *plot_emb_2d* è stata copiata dal notebook "VectorSemantics_fastText_UPDATE", il quale faceva parte dei materiali usati durante il primo modulo del Laboratorio di informatica umanistica. Quella successiva, invece, è stata creata per permettere di ottenere un plot a partire dai "vicini" di più parole, queste ultime inserite in una lista per poterla riutilizzare con un numero arbitrario di *input* (v. uso sotto).



In [ ]:
def plot_emb_2d(words, model):
  pca = PCA(n_components=2)
  vectors = np.array([model.get_word_vector(w) for w in words])
  #print(vectors)
  result = pca.fit_transform(vectors)
  fig = px.scatter(result, x=0, y=1, text=words, labels={"0":"Dimension 1","1":"Dimension 2"}, color_discrete_sequence=["red"], width=1200, height=600)
  fig.show()

In [ ]:
def visualise_neighbours_two(model, list_of_words, num_of_neighbours):

    all_neighbours = []

    for word in list_of_words:

        neighbours = model.get_nearest_neighbors(word, k=num_of_neighbours)

        list_of_neighbours = [neighbour[1] for neighbour in neighbours] #operazione necessaria per estrarre le parole e non il similarity score
        all_neighbours += list_of_neighbours

    plot = plot_emb_2d(all_neighbours, model)

    return plot


Pur potendo risultare in un primo momento abbastanza confusionario, osservando il risultato della rappresentazione 2D degli *embeddings* si possono ricostruire i gruppi delle parole appartenenti ai "vicini" di quelle passate come input nella lista di partenza:

In [ ]:
visualise_neighbours_two(first_period_model, trial_words, 5)

Avendo tuttavia l'intenzione di ricostruire i *cluster* delle parole semanticamente affini anche visivamente per facilitarne ulteriormente il riconoscimento, si è cercato di definire una funzione per renderli con colori differenti l'uno dall'altro. Si è seguito l'esempio proposto da Plotly e riportato [qui](https://stackoverflow.com/questions/60926439/plotly-add-traces-using-a-loop) da un utente di Stack Overflow. Per quel che riguarda l'uso di **graph_objects** al posto del solo metodo **scatter**, il cambio è stato reso necessario dal fatto che quest'ultimo sembra essere utilizzato per l'obiettivo qui fissato principalmente con DataFrame di pandas già completi ([qui](https://www.geeksforgeeks.org/python/python-plotly-how-to-set-up-a-color-palette/) un esempio). Inoltre, si è dovuto trovare un modo per evitare di ottenere una semplice sovrapposizione di plot tramite il calcolo della PCA a livello di parola singola e non globale.

Riassumendo ciò che la funzione fa:
1. Si inizializza un *graph object* di Plotly con **Figure**.
2. Si inizializza un "ciclo for" per ottenere tutti i "vicini" delle parole contenute nella lista *list_of_words* data come argomento della funzione. Si calcola quindi la PCA globale, il cui risultato è un array numpy di vettori a due dimensioni che sono la riduzione della rappresentazione orignaria a cento dimensioni delle parole.
3. Come unire la lista dei vicini ottenuta con fastText e quella di vettori ottenuti tramite l'applicazione della PCA? Essendo due liste in entrambi i casi, si potrebbe pensare di unirle, come suggerito in [questo tutorial di Geeks for Geeks](https://www.geeksforgeeks.org/python/create-a-dictionary-with-list-comprehension-in-python/), usando il metodo **zip**. Si è chiesto a ChatGPT di scrivere il codice che permettesse di farlo e di continuare nell'associazione delle parole e dei rispettivi "vicini" alle coordinate della PCA globale, oltre che al colore di riferimento per ciascun *cluster*. Da quello che si può vedere, il suggerimento dell'IA prevede di riottenere la lista di parole e "vicini" (qui usando un [segnaposto](https://towardsdatascience.com/why-python-loves-underscores-so-much-de03cf7bdcdd/) comune per avere solo uno dei valori di una tupla: l'underscore) per assegnare loro i valori delle "coordinate" della PCA contenute nel dizionario *word_to_coords*. Infine, il colore associato è quello presente nella posizione della lista *plot_colours_as_list* indicata dal risultato della divisione a risultato intero tra il valore attuale dell'indice e la lunghezza della lista di colori fornita, un modo effettivamente utile per evitare l'insorgenza di errori nel caso in cui i colori forniti siano insufficienti per il numero di parole di cui si stanno cercando i termini semanticamente affini.
4.  L'ultima parte è dedicata alla creazione e personalizzazione del plot: il [metodo **add_trace**](https://plotly.com/python/creating-and-updating-figures/) dentro al "ciclo for" definito dal codice suggerito da ChatGPT permette di aggiungere di volta in volta al _graph object_ inziale i punti che segnano le coordinate della PCA globale dei "vicini" di ciascuna parola. Gli assi sono definiti dalle due dimensioni associate a ciascuna parola; con **update_layout** sono stati personalizzati la grandezza e le descrizioni associate agli assi e al plot stesso.    

In [ ]:
def visualise_neighbours(model, list_of_words, num_of_neighbours, plot_colours_as_list):

        fig = go.Figure()
        all_neighbours = []

        for word in list_of_words:
            all_neighbours.append(word)
            neighbours = [neighbour_tuple[1] for neighbour_tuple in model.get_nearest_neighbors(word, k=num_of_neighbours)]
            all_neighbours += neighbours

        pca = PCA(n_components=2)
        vectors = np.array([model.get_word_vector(w) for w in all_neighbours])
        result = pca.fit_transform(vectors)

        word_to_coords = {word: coord for word, coord in zip(all_neighbours, result)} #INIZIO CHATGPT
        for index, word in enumerate(list_of_words):
            neighbours = [word] + [neighbour for _, neighbour in model.get_nearest_neighbors(word, k=num_of_neighbours)]
            coords = np.array([word_to_coords[n] for n in neighbours])
            colour = plot_colours_as_list[index % len(plot_colours_as_list)] #FINE CHATGPT

            fig.add_trace(go.Scatter(
                x=coords[:, 0],
                y=coords[:, 1],
                mode='markers+text',
                text=neighbours,
                marker=dict(size=10, color=colour),
                name=word
            ))

        fig.update_layout(
        title="Neighbours of Words in 2D PCA Space",
        xaxis_title="Dimension 1",
        yaxis_title="Dimension 2",
        width=1200,
        height=600
        )

        fig.show()

In [ ]:
list_of_colours = ["red", "blue", "purple", "green", "orange", "pink", "brown", "cyan", "magenta", "yellow"]

In [ ]:
visualise_neighbours(first_period_model, trial_words, 5, list_of_colours)

## L'analisi: 1700-1832

Iniziamo dalle basi: cosa dice il modello sulle affinità semantiche presenti in relazione agli aggettivi *womanly* e *manly*?

In [ ]:
first_period_model.get_nearest_neighbors("womanly")

[(0.8833332657814026, 'unwomanly'),
 (0.8344566822052002, 'womanliness'),
 (0.8153872489929199, 'maidenly'),
 (0.7482211589813232, 'femininely'),
 (0.7441542148590088, 'womanhood'),
 (0.7402470111846924, 'childlike'),
 (0.7334257364273071, 'comeliness'),
 (0.7289857268333435, 'lovable'),
 (0.7271808385848999, 'woman'),
 (0.7239573001861572, 'loveliness')]

In [ ]:
first_period_model.get_nearest_neighbors("manly")

[(0.7894183993339539, 'manliest'),
 (0.7586105465888977, 'manlier'),
 (0.7489627003669739, 'manlike'),
 (0.7485241293907166, 'manliness'),
 (0.7463738322257996, 'noble'),
 (0.745602548122406, 'undaunted'),
 (0.7426320314407349, 'resolute'),
 (0.7359726428985596, 'undauntedness'),
 (0.7288323640823364, 'modest'),
 (0.7249890565872192, 'undauntedly')]

Oltre alla ovvia apparizione di parole come *unwomanly* e *femininely*, si fanno notare nell’area della femminilità termini più interessanti come *childlike*, *comeliness* (“grazia”), *lovable* e *loveliness*. Apparentemente, tali risultati suggeriscono un’associazione tra femminilità e caratteristiche come la grazia, l’amabilità e, per la presenza dell’aggettivo childlike, l’innocenza e la mansuetudine.

Analizzando i “vicini” dell’aggettivo *manly*, invece, i risultati che appaiono, tralasciando anche qui le parole più ovvie, risaltano soprattutto caratteristiche come la “nobiltà” (*noble* ovviamente può avere anche connotazioni di tipo metaforico relative allo “spirito” di una persona), il coraggio e la risolutezza (*undaunted*, *resolute*), oltre alla modestia.

Pertanto, da queste primissime ricerche possiamo abbozzare i ritratti della femminilità e della mascolinità a partire dalle qualità che all’epoca identificavano positivamente tali concetti: la grazia, l’amabilità e l’innocenza per la prima; il coraggio, la risolutezza e la modestia per la seconda.


In [ ]:
visualise_neighbours(first_period_model, ["womanly", "manly"], 10, list_of_colours)

Ulteriori risultati a supporto di queste osservazioni si possono trarre dalle analogie del tipo “x sta a y come ___ sta a z”. Provando con l’analogia “manly is to man as ___ is to woman”, questi sono i risultati ottenuti:

In [ ]:
first_period_model.get_analogies("manly", "man", "woman")

[(0.7444022297859192, 'matronly'),
 (0.7132769227027893, 'unwomanly'),
 (0.7058130502700806, 'girl'),
 (0.6959239840507507, 'unfeminine'),
 (0.690112292766571, 'womanly'),
 (0.6867289543151855, 'artless'),
 (0.6837711334228516, 'modest'),
 (0.6817526817321777, 'matron'),
 (0.6778007745742798, 'girlish'),
 (0.666215181350708, 'graceful')]

La modestia, la grazia e l’educazione riappaiono, così come un tipo di innocenza fanciullesca, semplice e ingenua (*artless*). Similmente, l’analogia “womanly is to woman as ___ is to man” porta a questi risultati:

In [ ]:
first_period_model.get_analogies("womanly", "woman", "man")

[(0.6883869767189026, 'mind'),
 (0.6772482395172119, 'humanness'),
 (0.6677900552749634, 'nobleness'),
 (0.6651856899261475, 'singleness'),
 (0.6650057435035706, 'men'),
 (0.6627843379974365, 'feeling'),
 (0.6616256237030029, 'roundest'),
 (0.6604809761047363, 'handedness'),
 (0.6593970060348511, 'consciousnesses'),
 (0.657326877117157, 'souled')]

Ovvero, anche in questo caso si ritrovano qualità come la nobiltà (mi ritrovo a sostenere l’accezione di quest’ultima come relativa all’animo di una persona: v. *humanness* e *nobleness*) e la risolutezza (*singleness* e *roundest*). In alcuni casi i termini possono ovviamente essere polisemici e non avere un significato necessariamente corrispondente a quello da me dato. Il caso più evidente è quello di *singleness* che, tuttavia, anche nel periodo analizzato poteva significare “Sincerità, onestà, schiettezza, integrità; l’essere liberi da inganni, falsità e malizia”.

Da questo punto in avanti la ricerca sarà più specificamente legata a temi di particolare interesse e importanza. Come guida e riferimento è stato adottato il manuale di storia della letteratura *English Literature in Context* di Paul Poplawski, presente nelle bibliografie dei materiali di studio degli esami di letteratura inglese da me sostenuti durante il triennio e citata nella Bibliografia. A esso si aggiungeranno articoli e saggi che verranno usati per confrontarne tesi e risultati con quelli ottenuti dal modello.

Quando non diversamente indicato, si prenderanno altresì come riferimento le aree geografiche della Gran Bretagna e degli Stati Uniti, sia per gli argomenti affrontati che per le informazioni contenute nelle fonti sopracitate.

Infine, le ricerche dei significati di varie parole citate all’interno di questo documento sono state effettuate grazie all’[Oxford English Dictionary](https://www.oed.com/?tl=true) e alla possibilità che esso offre di analizzare esempi d’uso datati e i significati acquisiti nel tempo di più di 500.000 parole ed espressioni della lingua inglese («Oxford English Dictionary», s.d.).



### Identità e proprietà nel legame matrimoniale

La vita delle mogli del diciottesimo secolo era un continuo equilibrarsi tra obblighi, responsabilità e virtù. Modestia, castità e controllo erano le caratteristiche che più si ripetevano nella descrizione della donna virtuosa dell’epoca, sulle cui spalle pendeva anche la responsabilità di non gettare disonore sulla reputazione del marito (Begiato 2023, 2-3). Moglie e marito diventavano difatti un’unica entità in cui il potere del secondo soverchiava i diritti e i possibili “privilegi” della consorte nei diritti che egli aveva di gestire le sue proprietà terriere e nell’automatismo di diventare proprietario di tutti i suoi beni mobili. Di conseguenza, le donne sposate non possedevano una vera identità legale: contratti ed eventuali acquisti a credito dovevano essere stipulati e accordati dal marito, ovvero essere effettuati a suo nome. Tuttavia, in studi come quelli di Bailey (2002) si osserva come la *coverture*, ossia questo status di dipendenza e invisibilità che le donne sposate acquisivano in quanto obbligate a rivolgersi all’identità del marito nella gestione degli affari economici, all’epoca potesse pure essere considerata foriera di favoritismi nei confronti delle mogli per essere loro autorizzate a effettuare acquisti (sempre e comunque in nome del marito), a mantenere il controllo sulle proprietà ereditate qualora si manifestassero casi di violenza domestica e a essere “protette” dal nome del marito se avessero commesso furti in sua compagnia o se avessero contratto debiti (Bailey 2002, 351–52). Sempre Bailey afferma che “both married women and married men . . . conceptualized their contributions and property as pooled for familial and household benefit, rather than coming into the sole possession of the husband”. Pertanto, al di là di ciò che stabilivano le leggi, nelle coppie sposate si potevano creare delle dinamiche interessanti in merito alla stessa percezione di ciò che si considerava come “proprio”: negli stessi casi di separazione le donne potevano essere accusate di aver rubato oggetti che, per ragioni sia pratiche che emotive o di diritto sentito (per esempio per il fatto che appartenessero agli averi da loro introdotti nella proprietà familiare), percepivano come loro possedimenti nonostante le leggi che regolavano la *coverture* imponessero diversamente. Allo stesso modo e nello stesso tipo di casi, gli uomini erano inclini a considerare la proprietà da loro legalmente acquisita con il matrimonio nei loro beni e quelli delle mogli (363-66, 293, 295).

In Stone interessanti sono i passaggi in cui si presenta il dibattito sul tema dell’uguaglianza tra gli uomini secondo la legge di natura e come questa potesse o meno conciliarsi negli altri tipi di relazioni, come quelle familiari tra marito e moglie e padri e figli, a cui sottendevano gerarchie in cui era sempre l’uomo a occupare la posizione di maggior potere: in un periodo in cui l’assolutismo monarchico veniva messo costantemente in discussione e paragonato all’autorità patriarcale domestica, quest’ultima era sempre più al centro di critiche anche pungenti nei confronti delle sue forme più violente e asfissianti (Stone 1983, 265-67).

Quanto è possibile verificare la percezione generale di queste pratiche, leggi sulla proprietà e idee nei testi contenuti nel mio corpus? Di seguito si analizzeranno i risultati di analogie e affinità semantiche rilevate dal modello.



```
1: Matrimony is to husband as x is to wife / Matrimony is to wife as x is to husband
```

In [ ]:
first_period_model.get_analogies("matrimony", "husband", "wife", k = 15)

[(0.8035843968391418, 'matrimonial'),
 (0.7903302311897278, 'wedlock'),
 (0.7234535813331604, 'marriage'),
 (0.6830830574035645, 'marrieth'),
 (0.6797537803649902, 'marriageable'),
 (0.6777335405349731, 'unchristened'),
 (0.6744784712791443, 'marry'),
 (0.6678264141082764, 'christen'),
 (0.6579901576042175, 'batchelor'),
 (0.6541638970375061, 'goody'),
 (0.6499156355857849, 'cohabit'),
 (0.6463938355445862, 'unmarry'),
 (0.6440362930297852, 'cohabits'),
 (0.6415875554084778, 'banns'),
 (0.6409806609153748, 'widowhood')]

In [ ]:
first_period_model.get_analogies("matrimony", "wife", "husband", k = 15)

[(0.8375678658485413, 'matrimonial'),
 (0.7677380442619324, 'wedlock'),
 (0.7034935355186462, 'marriage'),
 (0.6753690838813782, 'elopements'),
 (0.6730032563209534, 'conjugal'),
 (0.6671497225761414, 'libertine'),
 (0.6611591577529907, 'courtship'),
 (0.6568254232406616, 'elopement'),
 (0.6521545052528381, 'marriageable'),
 (0.6516113877296448, 'marriages'),
 (0.6497284770011902, 'faulkland'),
 (0.6470093727111816, 'marrying'),
 (0.6442417502403259, 'courtships'),
 (0.6441686749458313, 'cohabits'),
 (0.6416544318199158, 'lovers')]

Dalle analogie appena formulate dal modello è interessante notare che nell’associazione del termine analogo alla parola “moglie” si ritrovano soprattutto parole genericamente riferite al matrimonio o ad alcuni suoi aspetti: *batchelor*, *widowhood* e, più distanti, *goody* (parola normalmente usata per rivolgersi in modo cortese a una donna sposata, a capo della gestione della casa o di attività alberghiere) e *banns* (avviso pubblico dato nelle chiese come annuncio di future unioni matrimoniali). Al contrario, i termini analoghi associati alla parola “marito” comprendono aspetti più “clandestini”, come le fughe organizzate da due amanti (*elopement*) e il libertinismo, oltre che il corteggiamento. È dunque possibile intuire, vista la natura perlopiù letteraria e non di cronaca dei testi che compongono il mio corpus, che fosse più accettabile associare tali aspetti ai mariti piuttosto che alle mogli. In Stone ciò viene confermato da citazioni tratte da *Advice to a Daughter*, saggio di Lord Halifax ripubblicato in diciassette edizioni prima del 1791, dove l'autore consiglia alla figlia di tollerare ed evitare la ribellione davanti ai possibili comportamenti indecenti del futuro marito tra i quali, appunto, il tradimento (1983, 309-10).  

```
2: Master is to man as x is to woman
```

In [ ]:
master_man_analogy = []
for tuple in first_period_model.get_analogies("master", "man", "woman", k = 20):
    master_man_analogy.append(tuple[1])

plot_emb_2d(master_man_analogy, first_period_model)

Da questa analogia le parole che appaiono identificano figure femminili in un contesto familiare, affaristico o lavorativo (*mistress*, *sister*, *daughter*, *tradeswoman*, *needlewoman*, *tailoress*...): il fatto che il modello indichi che queste ultime abbiano vettori molto simili rispetto all’analogia *master-man* di partenza potrebbe corroborare le osservazioni fatte sull'acquisizione di una certa agentività da parte delle donne in quanto incaricate della gestione della casa o impegnate a livello lavorativo. Tuttavia, risulta difficile stabilire l’effettiva correttezza di questa ipotesi data la presenza di parole che, al contrario, identificano lavori piuttosto umili come *maid*, *chambermaid*, *maidservant*, *waitress* e l'assenza del contesto vero e proprio in cui appaiono.

Il corrispettivo diretto di *master*, *mistress*, presenta lo stesso problema nel tentativo di associargli un significato più preciso rispetto al [ventaglio di quelli possibili che offre](https://www.oed.com/search/dictionary/?scope=Entries&q=mistress). Al contrario, sembra essere più semplice farlo nel caso di [*master*](https://www.oed.com/dictionary/master_n1?tab=meaning_and_use#37740038).


In [ ]:
first_period_model.get_nearest_neighbors("mistress", k = 15)

[(0.8543335199356079, 'maid'),
 (0.832511305809021, 'lover'),
 (0.8240075707435608, 'husband'),
 (0.8181344866752625, 'woman'),
 (0.8111203908920288, 'sister'),
 (0.804447591304779, 'mistresses'),
 (0.8044030666351318, 'waitress'),
 (0.8041551113128662, 'chambermaid'),
 (0.7944664359092712, 'lady'),
 (0.7938212156295776, 'confidante'),
 (0.790749192237854, 'daughter'),
 (0.7847582101821899, 'mother'),
 (0.7776350378990173, 'traitress'),
 (0.7739921808242798, 'governess'),
 (0.7705366611480713, 'princess')]

In [ ]:
first_period_model.get_nearest_neighbors("master", k = 15)

[(0.8046191334724426, 'masterman'),
 (0.7805580496788025, 'masters'),
 (0.7764692902565002, 'masterson'),
 (0.7652771472930908, 'headmaster'),
 (0.7573290467262268, 'sent'),
 (0.7551928758621216, 'man'),
 (0.7547533512115479, 'grandmaster'),
 (0.7526047229766846, 'commanded'),
 (0.746705949306488, 'pinguister'),
 (0.743178129196167, 'shipmaster'),
 (0.7425553202629089, 'command'),
 (0.7418785095214844, 'mates'),
 (0.7414924502372742, 'waring'),
 (0.7411244511604309, 'went'),
 (0.7336364984512329, 'king')]

```
3: Workman/workingman is to man as x is to woman
```

In [ ]:
first_period_model.get_analogies("workman", "man", "woman", k = 15)

[(0.7145335078239441, 'workwoman'),
 (0.6729543805122375, 'journeyman'),
 (0.640800416469574, 'workmanlike'),
 (0.6333250403404236, 'porkman'),
 (0.6291452050209045, 'wigmaker'),
 (0.6284466981887817, 'girl'),
 (0.62699294090271, 'brickmaker'),
 (0.6266695857048035, 'clockmaker'),
 (0.6253727674484253, 'needlewoman'),
 (0.6217162609100342, 'locksmiths'),
 (0.6161882877349854, 'tradeswoman'),
 (0.6150496602058411, 'locksmith'),
 (0.6132596731185913, 'tailor'),
 (0.6102455854415894, 'manufacturer'),
 (0.6036479473114014, 'loom')]

Si ritrovano anche qui *needlewoman* e *tradeswoman* tra le parole affini più significative. È altrettanto interessante che, tra le altre sette presenti, quelle attinenti alle attività lavorative domestiche come *chambermaid* e *nanny* (“donna incaricata della pulizia di camere da letto o domestica al servizio di una lady o di un’altra donna”; “tata”) non appaiono più, mentre sono in netta maggioranza le parole che fanno più specificamente riferimento a figure professionali "maschili". È altresì giusto precisare che tali occorrenze, essendo *workman* riferito a lavoratori di bassa o media estrazione sociale impiegati in mansioni che richiedono una certa fatica fisica, confermerebbero la presenza di un *bias* nei risultati dell'analogia "master-man-woman". A riprova di ciò, con una semplice funzione sono stati recuperati i contesti di origine di tali parole:

```
def find_context(directory, length, given_word):

  contexts_counter = 0

  for filename in tqdm(glob.iglob(f'{directory}/*'), total = length, desc = "Finding contexts with the given word:"):
    with open(filename, "r") as f:
        lines = f.readlines()
        for index, line in enumerate(lines):
          if given_word in line:
            contexts_counter += 1
            print(f"CONTEXT NUMBER {contexts_counter}, FOUND IN {filename}:\n\n {line}")
            if index + 1 < len(lines):
              print(f"{lines[index+1]}")
            if index +2 < len(lines):
              print(f"{lines[index+2]}\n---\nEND OF CONTEXT\n")
            else:
              print("---\nEND OF CONTEXT\n")
```

La funzione ha permesso di "stampare" un determinato numero di righe (delimitate da caratteri *newline* "\n") dei documenti originali dei due corpora ogniqualvolta una certa parola appariva negli stessi. Qui è stata resa in *markdown* perché usata originariamente in un notebook di prova di Colab che ne permetteva un uso diretto per essere già collegato alla cartella Drive che contiene i testi dentro cui viene effettuata la ricerca. Al contrario, nell'ambiente virtuale di Visual Studio Code si sarebbe reso necessario del codice aggiuntivo per l'autenticazione e l'accesso tramite credenziali Google alla stessa.

I risultati ottenuti dall'analogia "master-man-woman" sono così stati usati per cercare di estrarre soprattutto elementi valorativi utili a confermare l'ipotesi, generata dall'osservazione dei valori di *cosine similarity* ottenuti dal modello, dell'esistenza di una forma di discriminazione nell'associazione della donna a lavori manuali e professioni umili in contrasto con il quasi inequivoco carattere di "padrone" del maschile *master*:

**NEEDLEWOMAN:**

"My lieutenant went to get what I wanted, and particularly a **needlewoman** to make me some shirts. In the course of the day I had furniture, bedding, kitchen utensils, a good dinner, twenty-four well-equipped soldiers, a super-annuated sempstress and several young girls to make my shirts. After supper, I found my position highly pleasant, being surrounded with some thirty persons who looked upon me as their sovereign, although they could not make out what had brought me to their island. The only thing which struck me as disagreeable was that the young girls could not speak Italian, and I did not know Greek enough to enable me to make love to them."

"We had a new landlady from Orleans; she took for a **needlewoman** a girl from her own country, of between twenty--two and twenty--three years of age, and who, as well as the hostess, ate at our table. This girl, named . . . "

**TAILORESS**

" . . . had commissioned a certain **tailoress** to make me a waistcoat. Meeting me in the road, this woman said that she had lost my measure, and asked whether I would come that evening and let her measure me again. I went, . . . "

" . . . without speaking about my waistcoat to the **tailoress**. A few months after this interview she married the merchant. I saw her occasionally in the street together with her husband. She was always beautiful. On . . . "

" . . . **tailoress**, ogling and flirting over the commission, while a running footman with tall cane in hand comes bustling in to ask if his lord's suit is ready; the old patrician lolling in his easy-chair and toying . . . "

" . . . represents exactly such a scene as this in the workroom of a **tailoress**. The beau is there, and the woman prepared for flirtation."


**TRADESWOMAN**

" . . . **tradeswoman** behind a counter, whose looks were so charmingly attractive, that, notwithstanding my timidity with the ladies, I entered the shop without hesitation, offered my services as usual: and had the happiness . . . "

" . . . covered her neck like a **tradeswoman**.  "O, for that matter," replied the person she was speaking to (who was fond of a joke), "she has good reason, for I know she is marked with a great ugly rat on her bosom, so . . . "

" . . . her like a **tradeswoman**? She would teach him to respect women of her birth; said he was in league with Mr. Sheffield (377) to abuse her, and bade him come the next morning at nine." He came, . . . "

---

Per completezza, specifico che usando la funzione per il recupero di questi estratti dai testi originali del periodo 1700-1831 ne ho trovati alcuni duplicati ma provenienti da opere contrassegnate con un ID diverso; lo stesso non si è verificato cercando tra i testi del periodo 1832-1932. Il problema non è dovuto alle diverse modalità di reperimento degli ID, bensì a edizioni diverse delle stesse opere: questo è coerente con il risultato di una prova effettuata cercando con pandas tra gli ID dello stesso catalogo del *PG* utilizzato nel notebook precedente.

Da tutti questi esempi (e da una ricerca più approfondita sfruttando gli ID dei testi in cui sono stati trovati) la riflessione che se ne ricava è che, perlomeno nel discorso pubblico, tali figure professionali femminili non godessero di prestigio: nelle opere letterarie del corpus del primo periodo in cui appaiono sono oggetto delle attenzioni amorose e sessuali di narratori uomini, semplici testimoni di incontri tra un narratore e la sua amante, parte della servitù di persone di potere o, cosa ancora più interessante, simbolicamente legate a caratteristiche negative in una data cultura: il penultimo estratto qui proposto proviene dalle *Confessioni* di Rousseau e si riferisce a un pudore suppostamente esagerato mostrato da una nobildonna nel coprirsi il petto; l'ultimo dalle *Letters of Horace Walpole* e, anche senza aggiungere ulteriori dettagli alla situazione lì descritta (se non il fatto che la frase venga pronunciata da una duchessa), già la successione delle frasi lascia intuire come un trattamento da *tradeswoman* non fosse proprio considerato desiderabile.

Pertanto, per quanto semplice possa essere un approccio di questo tipo, le analogie possono essere usate per ipotizzare la presenza di pregiudizi all'interno dei testi dei corpora e confermare l'esistenza di tali *bias* in altre.

Infine, riflettendo sulla possibile differenza esistente nella vicinanza semantica tra termini come *owner* o *proprietor* e quelli relativi a “uomo”/ “donna” e “moglie”/ “marito”, sono state fatte delle prove misurando la *cosine similarity* tra le varie coppie, senza che però venissero riscontrate differenze significative.

La funzione per il calcolo della *cosine_similarity* è stata anch'essa copiata dal già menzionato notebook presente tra i materiali del Laboratorio avanzato di informatica umanistica del terzo anno. Quella successiva l'ho creata per poter ottenere come risultato le due parole confrontate e il valore della *cosine_similarity*.

In [ ]:
# Function to compute cosine similarity
def cosine_similarity(vec1, vec2):
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

In [ ]:
def compute_similarity_on_tuples(list_of_tuples, model):

    list_of_similarities = []

    for tuple in list_of_tuples:
        word1 = tuple[0]
        word2 = tuple[1]

        vec1 = model.get_word_vector(word1)
        vec2 = model.get_word_vector(word2)

        similarity = cosine_similarity(vec1, vec2)
        print("Cosine Similarity between '{}' and '{}' is {}".format(word1, word2, similarity))

        list_of_similarities.append((word1, word2, similarity))

    return list_of_similarities

In [ ]:
tuples_for_sim_scores_on_writing = [("owner", "woman"), ("owner", "man"), ("owner", "wife"), ("owner", "husband"), ("proprietor", "wife"), ("proprietor", "husband"), ("proprietor", "man"), ("proprietor", "woman")]
compute_similarity_on_tuples(tuples_for_sim_scores_on_writing, first_period_model)

Cosine Similarity between 'owner' and 'woman' is 0.594893753528595
Cosine Similarity between 'owner' and 'man' is 0.5986189842224121
Cosine Similarity between 'owner' and 'wife' is 0.5486278533935547
Cosine Similarity between 'owner' and 'husband' is 0.5166386961936951
Cosine Similarity between 'proprietor' and 'wife' is 0.46964481472969055
Cosine Similarity between 'proprietor' and 'husband' is 0.42044010758399963
Cosine Similarity between 'proprietor' and 'man' is 0.46102175116539
Cosine Similarity between 'proprietor' and 'woman' is 0.44142061471939087


[('owner', 'woman', np.float32(0.59489375)),
 ('owner', 'man', np.float32(0.598619)),
 ('owner', 'wife', np.float32(0.54862785)),
 ('owner', 'husband', np.float32(0.5166387)),
 ('proprietor', 'wife', np.float32(0.4696448)),
 ('proprietor', 'husband', np.float32(0.4204401)),
 ('proprietor', 'man', np.float32(0.46102175)),
 ('proprietor', 'woman', np.float32(0.4414206))]

### Il mestiere e le forme letterarie della scrittrice

Durante l’ultima parte di questo primo periodo, ovvero quella che corrisponde al Romanticismo, le scrittrici si potevano già contare sulle migliaia. Ciononostante, vigevano ancora standard e criteri, definiti e promossi da editori e periodici, su cosa esse potessero effettivamente scrivere. In aggiunta, con l’avvento dello sviluppo del romanzo e della sua maggiore diffusione, la poesia e l’epica divennero forme “maschili”, sia per lo sbilanciamento nel numero di autori uomini che componevano in versi, sia per il tipo di educazione e gusto che la poesia richiedeva. Al contrario, lo stesso romanzo divenne una forma piuttosto usata dalle donne, le quali erano propense a trattare argomenti legati all’ambiente domestico e familiare (Kitson 2017, 323-24).  

I risultati più interessanti per questa parte sono stati riscontrati misurando la *cosine similarity* tra varie parole attinenti:


In [ ]:
compute_similarity_on_tuples([("writer", "woman"), ("writer", "man"), ("poet", "man"), ("poet", "woman"), ("novelist", "woman"), ("novelist", "man")], first_period_model)

Cosine Similarity between 'writer' and 'woman' is 0.4905639588832855
Cosine Similarity between 'writer' and 'man' is 0.6125530004501343
Cosine Similarity between 'poet' and 'man' is 0.5203648805618286
Cosine Similarity between 'poet' and 'woman' is 0.45095688104629517
Cosine Similarity between 'novelist' and 'woman' is 0.5175894498825073
Cosine Similarity between 'novelist' and 'man' is 0.42237338423728943


[('writer', 'woman', np.float32(0.49056396)),
 ('writer', 'man', np.float32(0.612553)),
 ('poet', 'man', np.float32(0.5203649)),
 ('poet', 'woman', np.float32(0.45095688)),
 ('novelist', 'woman', np.float32(0.51758945)),
 ('novelist', 'man', np.float32(0.42237338))]

Sembra, anche se per distacchi in alcuni casi abbastanza risicati, che questi risultati confermino quanto scritto sopra.

## Secondo periodo (1832-1932)

In questa sezione ci si dedicherà all'analisi delle affinità semantiche riscontrate nei risultati dati dal modello allenato per il secondo periodo di riferimento del progetto (1832-1932). Si seguirà lo stesso filo di argomenti, iniziando anche qui dalle basi: cosa ci dice questo modello sugli aggettivi *womanly* e *manly*?

In [ ]:
second_period_model.get_nearest_neighbors("womanly", k = 15)

[(0.8709399700164795, 'unwomanly'),
 (0.8627868294715881, 'womanliness'),
 (0.8486061096191406, 'womanlike'),
 (0.8435689806938171, 'femininely'),
 (0.8336664438247681, 'unwomanliness'),
 (0.8236613869667053, 'tenderness'),
 (0.8194234371185303, 'feminine'),
 (0.8069148659706116, 'girlish'),
 (0.79427570104599, 'unfeminine'),
 (0.7896934151649475, 'girlishness'),
 (0.7890847325325012, 'passionateness'),
 (0.7859819531440735, 'femininity'),
 (0.7853972911834717, 'maidenly'),
 (0.785274088382721, 'childlike'),
 (0.7705596089363098, 'maidenliness')]

In [ ]:
second_period_model.get_nearest_neighbors("manly", k = 15)

[(0.7655788064002991, 'manliness'),
 (0.758406400680542, 'womanly'),
 (0.7579648494720459, 'gentlemanliness'),
 (0.7534784078598022, 'manlier'),
 (0.74275141954422, 'courageous'),
 (0.7410263419151306, 'manliest'),
 (0.7409602999687195, 'sprightliness'),
 (0.7351250648498535, 'brave'),
 (0.7320507764816284, 'womanlike'),
 (0.728811502456665, 'manlike'),
 (0.72640460729599, 'courage'),
 (0.7225343585014343, 'womanliness'),
 (0.7216212153434753, 'warmness'),
 (0.7204902172088623, 'heartiness'),
 (0.7181591391563416, 'noble')]

In [ ]:
visualise_neighbours(second_period_model, ["womanly", "manly"], 15, list_of_colours)

L'unica differenza con il periodo anteriore sembra essere l'aggiunta di *passionateness* e *sprightliness* ("vivacità", "brio", "vigore"...) ai "vicini" delle due parole scelte. È ad ogni modo interessante che anche delle parole appartenenti al *cluster* di *manly* (come *womanly* e *womanliness*) si ritrovino tra quelle del *cluster* di *womanly*: ciò indica ancora il buon funzionamento del modello allenato.

Possiamo andare più a fondo nell'analisi delle parole semanticamente più affini a *womanly*?

In [ ]:
second_period_model.get_nearest_neighbors("unfeminine", k = 40)

[(0.8660288453102112, 'feminine'),
 (0.842370331287384, 'femininely'),
 (0.8104749321937561, 'unwomanly'),
 (0.7963306307792664, 'unwomanliness'),
 (0.7942755818367004, 'womanly'),
 (0.7810783982276917, 'femininity'),
 (0.772331714630127, 'womanish'),
 (0.762724757194519, 'masculine'),
 (0.7591524720191956, 'masculinely'),
 (0.7542673349380493, 'womanliness'),
 (0.7510169744491577, 'clownishness'),
 (0.7489719986915588, 'masculinity'),
 (0.7477750182151794, 'maidenliness'),
 (0.7467008233070374, 'unmaidenly'),
 (0.7452419996261597, 'womanlike'),
 (0.7400570511817932, 'childishness'),
 (0.7385407090187073, 'shrewishness'),
 (0.738131046295166, 'fastidiousness'),
 (0.7356148362159729, 'prudishness'),
 (0.7328760027885437, 'artlessness'),
 (0.7320887446403503, 'unmanly'),
 (0.7294526100158691, 'artless'),
 (0.7279939651489258, 'slavishness'),
 (0.7277995347976685, 'boorishness'),
 (0.7274839282035828, 'womankind'),
 (0.7252275943756104, 'sexlessness'),
 (0.7240809202194214, 'pettishness')

In [ ]:
visualise_neighbours(second_period_model, ["unfeminine"], 40, list_of_colours)

Provando a cercare ciò che non era da considerarsi “femminile” nel periodo analizzato le parole che più saltano all’occhio nell’insieme risultante di affinità semantiche sono, ovviamente, quelle che più intuitivamente fanno riferimento alle caratteristiche contrarie a quanto rilevato precedentemente per gli aggettivi *womanly* e *manly*. La presenza di parole che palesemente non si riferiscono a qualità contrarie alla femminilità (come *womanly*, *maideliness*, ecc.) credo siano da interpretarsi come conseguenza naturale del fatto che possano trovarsi molto spesso negli stessi contesti di parole contrarie.

Tralasciando quindi questi “intrusi”, troviamo caratteristiche come *clownishness*, *shrewishness* (“irritabile”, “bisbetico”), *fastidiousness*, *boorishness* (“dozzinalità”, “grossolanità”) e altre che, seguendo perlomeno i precetti espliciti che soprattutto nell’epoca vittoriana si davano alle giovani donne, non sorprende che appaiano (Frawley 2017, 433-34). Molto chiara, in tal senso, è l’implicazione contenuta nel primo capitolo di *Jane Eyre* di Charlotte Brontë, dove la protagonista, ancora bambina, viene obbligata al silenzio e all’isolamento, almeno fino a quando non riuscirà a contenere un atteggiamento visto come troppo insolente e poco amichevole da Mrs Reed, sua zia e tutrice legale. Nelle stesse pagine Jane è costretta a reagire bruscamente a un attacco fisico da parte del cugino John e sarà l’unica dei due a pagarne le conseguenze, ancora una volta con l’isolamento, essendo rinchiusa nella *red-room* (Brontë 1999, 3-7).

Anche in Stone si legge di come l'ideale femminile vittoriano imponesse alle donne una sottomissione assoluta alle convenzioni e al marito. Nei romanzi di Jane Austen l'allocutivo "Mr" usato sia in privato col marito sia nei rapporti sociali con uomini non appartenenti al proprio nucleo familiare può essere visto come esempio letterario di queste imposizioni (1983, 155-56).

### Alcune caratteristiche della donna vittoriana

Alla misura e alla modestia promossi dal moralismo vittoriano come attributo desiderabile nelle donne viene associata la dicotomia che contrapponeva le stesse nelle figure, rispettivamente, dell’angelo del focolare e della *fallen woman* (o della donna tentatrice e infamata a causa di una condotta sessuale biasimata dall’opinione pubblica) (Frawley 2017, 435-36). Altri standard molto restrittivi per le donne erano rappresentati dall’aspetto fisico ideale che veniva loro imposto: busto e fianchi ben sviluppati e messi in risalto da un girovita molto contenuto e modellato da bustini strettissimi (Shuttleworth 1997, 83). Effettuando alcune prove con la *cosine similarity* e parole relative a questi fenomeni si è portati a intuire che l’attenzione nei confronti di questi aspetti fosse effettivamente maggiore in quei contesti in cui venivano menzionate le donne.

In [ ]:
compute_similarity_on_tuples([("seductress", "woman"), ("shameful", "woman"), ("shameful", "man"), ("sinful", "woman"), ("sinful", "man"), ("slim", "woman"), ("slim", "man"), ("slimness", "woman"), ("slimness", "man"), ("slender", "woman"), ("slender", "man")], second_period_model)

Cosine Similarity between 'seductress' and 'woman' is 0.6253945231437683
Cosine Similarity between 'shameful' and 'woman' is 0.5658844709396362
Cosine Similarity between 'shameful' and 'man' is 0.3624839782714844
Cosine Similarity between 'sinful' and 'woman' is 0.5731052756309509
Cosine Similarity between 'sinful' and 'man' is 0.3915311396121979
Cosine Similarity between 'slim' and 'woman' is 0.5575359463691711
Cosine Similarity between 'slim' and 'man' is 0.3135552704334259
Cosine Similarity between 'slimness' and 'woman' is 0.517393171787262
Cosine Similarity between 'slimness' and 'man' is 0.3480556309223175
Cosine Similarity between 'slender' and 'woman' is 0.5198826789855957
Cosine Similarity between 'slender' and 'man' is 0.402288556098938


[('seductress', 'woman', np.float32(0.6253945)),
 ('shameful', 'woman', np.float32(0.5658845)),
 ('shameful', 'man', np.float32(0.36248398)),
 ('sinful', 'woman', np.float32(0.5731053)),
 ('sinful', 'man', np.float32(0.39153114)),
 ('slim', 'woman', np.float32(0.55753595)),
 ('slim', 'man', np.float32(0.31355527)),
 ('slimness', 'woman', np.float32(0.5173932)),
 ('slimness', 'man', np.float32(0.34805563)),
 ('slender', 'woman', np.float32(0.5198827)),
 ('slender', 'man', np.float32(0.40228856))]

La magrezza ideale si accompagnava a una generale idea normativa della donna come creatura estremamente fragile e, come si legge in Stone, ciò portava a delle vere e proprie manifestazioni psicosomatiche di stati di debolezza e malattia: le più giovani, assimilando il modello culturale che le veniva imposto, arrivavano a credere di essere colpite da vari tipi di malessere. Tra tutti, l'isteria e l'ipocondria si consideravano come le malattie più diffuse tra le donne a causa della prevalenza del sentimento e dell'immaginazione sul raziocinio (1983, 763).

Non è allora un caso che tra le parole semanticamente affini risultanti dall'analogia "malady-man-woman" appaiano varie volte parole come *neurotic* e *hypochondriasis*, anche nelle loro forme flesse.

In [ ]:
second_period_model.get_analogies("malady", "man", "woman", k = 20)

[(0.6916494965553284, 'neuralgia'),
 (0.6709731221199036, 'illness'),
 (0.6491451263427734, 'neurotic'),
 (0.6448416113853455, 'anaemia'),
 (0.6386134028434753, 'maladies'),
 (0.6356241106987, 'hypochondriasis'),
 (0.6316859722137451, 'illnesses'),
 (0.629937469959259, 'neuralgic'),
 (0.6299342513084412, 'dropsy'),
 (0.6239355802536011, 'infantile'),
 (0.6235175728797913, 'symptoms'),
 (0.6229698061943054, 'sickness'),
 (0.6181862354278564, 'anæmia'),
 (0.6179444789886475, 'neuritis'),
 (0.614641010761261, 'maladie'),
 (0.6120611429214478, 'ailment'),
 (0.6107059717178345, 'chronic'),
 (0.6062659025192261, 'nymphomania'),
 (0.604982316493988, 'bronchitis'),
 (0.6032238006591797, 'peripneumonia')]

Curiosamente, fa la sua apparizione anche la parola *nymphomania*, che riappare, in maniera significativa insieme a *neurotic*, in solo una delle analogie sottostanti:

In [ ]:
second_period_model.get_analogies("sexual", "woman", "man", k = 20)

[(0.7330942153930664, 'sexually'),
 (0.7045595049858093, 'coordination'),
 (0.6793113350868225, 'coördination'),
 (0.6681203246116638, 'sexuality'),
 (0.6461270451545715, 'conduces'),
 (0.6454916596412659, 'resulting'),
 (0.6425209045410156, 'moral'),
 (0.6421037912368774, 'abstention'),
 (0.6347184181213379, 'functional'),
 (0.6343936920166016, 'coordinate'),
 (0.6334124207496643, 'mental'),
 (0.6295561194419861, 'inherent'),
 (0.6258475184440613, 'reactions'),
 (0.6247875094413757, 'stimulation'),
 (0.6239831447601318, 'ideational'),
 (0.6231130361557007, 'coördinate'),
 (0.6221295595169067, 'coordinated'),
 (0.6217362880706787, 'congenital'),
 (0.6164171695709229, 'functions'),
 (0.6141961812973022, 'interactions')]

In [ ]:
second_period_model.get_analogies("sexual", "man", "woman", k = 20)

[(0.7330382466316223, 'neurotic'),
 (0.7203279137611389, 'sexuality'),
 (0.7161739468574524, 'sexually'),
 (0.6988025307655334, 'erotic'),
 (0.6720712780952454, 'neurosis'),
 (0.6580895781517029, 'nymphomania'),
 (0.656193196773529, 'sex'),
 (0.6345266103744507, 'womanish'),
 (0.6305017471313477, 'spermatorrhœa'),
 (0.6183549165725708, 'womanhood'),
 (0.6146417856216431, 'womankind'),
 (0.6126750707626343, 'neuroses'),
 (0.6123335957527161, 'sexlessness'),
 (0.6095536947250366, 'myopia'),
 (0.6062755584716797, 'aphasia'),
 (0.6022786498069763, 'abnormal'),
 (0.5950663089752197, 'femaleness'),
 (0.5943595170974731, 'eroticism'),
 (0.5939137935638428, 'femininity'),
 (0.5938843488693237, 'morbid')]

Questa relazione tra instabilità emotiva e sessualità femminile riapparirà nella successiva sezione di analisi di alcune differenze semantiche rilevate tra i periodi. A completamento di quella, si rimanda sempre a Stone e alla descrizione dell'atteggiamento, anche qui ideale, della donna nei confronti del desiderio sessuale: freddo e distaccato per non comprometterne la salute delicata. La lussuria era riservata agli uomini, mentre le donne potevano addirittura essere considerate quasi come degli esseri asessuali in cui il desiderio spesso non si manifestava (1983, 763-64).

### Femminismo

Il secolo preso in considerazione per questo periodo è particolarmente rilevante per lo sviluppo della figura letteraria della “New Woman” e per la centralità che arrivarono ad acquisire temi come l’affrancamento della donna dall’opprimente sfera domestica e dalle altrettanto limitanti condizioni a essa imposte dal matrimonio (Frawley 2017, 402). Verso la fine del XIX secolo e nei primi decenni di quello successivo vennero fondate organizzazioni femministe che ambivano all’uguaglianza politica e a pari opportunità lavorative tra uomini e donne (Frawley 2017, 512-13; Franklin 2003, 106-7).

Ho provato a cercare tra i vicini delle parole più caratteristiche legate alle partecipanti di queste organizzazioni e movimenti, *sufragettes* e *suffragists*, per capire se ci fossero differenze nel tipo di significato veicolato dalle stesse e, una volta stabilito che stando al modello non ce n'erano, ho analizzato i termini affini associati alla seconda.

In [ ]:
second_period_model.get_nearest_neighbors("suffragists", k = 20)

[(0.90921950340271, 'suffragist'),
 (0.8972421884536743, 'suffrages'),
 (0.8951894640922546, 'suffrage'),
 (0.7925878763198853, 'legitimists'),
 (0.7890341877937317, 'constitutionalists'),
 (0.775965690612793, 'advocates'),
 (0.7708948254585266, 'majorities'),
 (0.7687903046607971, 'suffragettes'),
 (0.7680969834327698, 'majority'),
 (0.7617713809013367, 'democratic'),
 (0.7591494917869568, 'alienists'),
 (0.7570192217826843, 'minority'),
 (0.7565454244613647, 'democracies'),
 (0.7544870972633362, 'minorities'),
 (0.7535760998725891, 'socialists'),
 (0.7529556751251221, 'undemocratic'),
 (0.7524240016937256, 'communists'),
 (0.7499695420265198, 'individualists'),
 (0.7484340071678162, 'enfranchisement'),
 (0.748344898223877, 'institutionalized')]

Ci sono risultati più semplici da spiegare di altri (*enfranchisement*, *socialist*), con l'apparizione interessante di *alienists* (psichiatri)... Non mi spingo più a fondo in una possibile giustificazione della sua presenza, se non per il momento allegando [questo articolo dall'archivio del *New York Times*](https://www.nytimes.com/1909/11/14/archives/militant-suffragist-madness.html?smid=url-share) (datato 14 novembre 1909) in cui appare chiaramente un'associazione tra femminismo (soprattutto nelle sue manifestazioni più radicali) e instabilità mentale.

## Analisi del cambio semantico



Tra le parole analizzate per ciascun periodo è possibile commentare qualche cambiamento occorso nel significato e nelle affinità semantiche rilevate dai modelli?

In [ ]:
first_period_model.get_nearest_neighbors("woman", k=15)

[(0.818134605884552, 'mistress'),
 (0.7981212735176086, 'husband'),
 (0.7956501841545105, 'gentlewoman'),
 (0.7927596569061279, 'girl'),
 (0.7923548817634583, 'maiden'),
 (0.7835094928741455, 'maid'),
 (0.7690948843955994, 'lover'),
 (0.7671993374824524, 'wife'),
 (0.7601224780082703, 'child'),
 (0.7570766806602478, 'young'),
 (0.7557933330535889, 'lady'),
 (0.7528080344200134, 'countrywoman'),
 (0.7522703409194946, 'doatingly'),
 (0.751965343952179, 'daughter'),
 (0.7519515752792358, 'mother')]

In [ ]:
second_period_model.get_nearest_neighbors("woman", k = 15)

[(0.8426957130432129, 'girl'),
 (0.8141846060752869, 'husband'),
 (0.7971101403236389, 'wife'),
 (0.7879599332809448, 'child'),
 (0.7745593190193176, 'daughter'),
 (0.7726542353630066, 'young'),
 (0.7703341245651245, 'freedwoman'),
 (0.765163004398346, 'wooman'),
 (0.7597655057907104, 'mother'),
 (0.7558220028877258, 'madwoman'),
 (0.7466315627098083, 'daughterly'),
 (0.7446277141571045, 'creature'),
 (0.7386703491210938, 'lady'),
 (0.7382584810256958, 'lover'),
 (0.7361214756965637, 'poor')]

Un aspetto interessante nel passaggio dal primo al secondo periodo riguarda proprio la parola *woman*, che acquisisce un “vicino” particolarmente significativo, soprattutto se lo si interpreta alla luce degli studi sulla letteratura vittoriana: *madwoman*. Questo termine si può interpretare non solo analizzando le figure femminili che più la rappresentano nella letteratura gotica e in quella successiva (v. la [“*madwoman in the attic*”](https://en.wikipedia.org/wiki/The_Madwoman_in_the_Attic) in  *Jane Eyre* di Charlotte Brontë), bensì anche nei rilevamenti statistici dell’epoca sul numero di pazienti donne all’interno degli ospedali psichiatrici: verso la fine del XIX secolo esse erano in maggioranza rispetto ai pazienti uomini nella gran parte di queste strutture (Showalter 1987, 52). Le donne si consideravano più vulnerabili emotivamente e psicologicamente a causa di una supposta instabilità nel ciclo di vita rappresentato, nel loro caso, dalla successione delle fasi della pubertà, della gestazione e del parto, e della menopausa; in breve, un ciclo determinato dal loro apparato riproduttivo (55). Nel commento di Elaine Showalter a *Mrs. Dalloway* di Virginia Woolf (pubblicato originariamente nel 1924) si mettono in relazione l’età della protagonista, una signora sulla cinquantina, le intenzioni dell’autrice nella sua descrizione prendendo in considerazione proprio quell’età e l’inizio della medicalizzazione della menopausa già a partire dalla metà del secolo precedente alla scrittura del romanzo: l'idea che l'instabilità del ciclo di vita determinato dall'apparato riproduttivo femminile generasse degli squilibri a livello mentale e che, nel caso specifico della menopausa, potesse avere a che fare con episodi di depressione e suicidio persistette anche oltre la fine dell’epoca vittoriana (Showalter 1992, 33).    

Non sorprende quindi che anche nelle parole affini a *suffragist* apparisse la figura dello psichiatra: al di là della rappresentazione eccessivamente restrittiva di un modello femminile quasi robotico nella compostezza e negli standard elevati che le donne dovevano mantenere, come ribadito da Showalter questa inclinazione "naturale" delle donne allo squilibrio psichico le precludeva l'accesso a diritti fondamentali come quello di voto, di libero accesso a varie professioni e di affrancamento della propria identità dai vincoli familiari (73).

Anche le stesse parole *suffragists* e *feminists* rappresentano un cambiamento, dalla vaghezza e incorrettezza dei risultati ottenuti per il primo periodo a quelli più precisi per il secondo. Ciò riflette i controlli effettuati nell’*OED* sulle stesse: la prima occorrenza di *suffragist* risale al 1818 nell’espressione *universal suffragist*, al 1871 nella più specifica *woman suffragist*; la prima occorrenza di *feminist*, invece, è datata al 1852 (Oxford English Dictionary 2024b; 2024a).


In [ ]:
visualise_neighbours(first_period_model, ["feminists", "suffragists"], 20, list_of_colours)

In [ ]:
visualise_neighbours(second_period_model, ["feminists", "suffragists"], 20, list_of_colours)

Provando a misurare la *cosine similarity* per quel che riguarda una delle caratteristiche del fisico femminile principalmente promosse nell'epoca vittoriana, invece, la differenza tra i due periodi è sì presente, ma nel primo periodo appare meno importante in termini di differenze tra i valori di somiglianza relativi alle coppie di ciascun periodo:

In [ ]:
compute_similarity_on_tuples([("slim", "woman"), ("slim", "man"), ("slimness", "woman"), ("slimness", "man"), ("slender", "woman"), ("slender", "man")], first_period_model)

Cosine Similarity between 'slim' and 'woman' is 0.5522605776786804
Cosine Similarity between 'slim' and 'man' is 0.3678361177444458
Cosine Similarity between 'slimness' and 'woman' is 0.4413400888442993
Cosine Similarity between 'slimness' and 'man' is 0.3978479504585266
Cosine Similarity between 'slender' and 'woman' is 0.4369645118713379
Cosine Similarity between 'slender' and 'man' is 0.4722546935081482


[('slim', 'woman', np.float32(0.5522606)),
 ('slim', 'man', np.float32(0.36783612)),
 ('slimness', 'woman', np.float32(0.4413401)),
 ('slimness', 'man', np.float32(0.39784795)),
 ('slender', 'woman', np.float32(0.4369645)),
 ('slender', 'man', np.float32(0.4722547))]

In [ ]:
compute_similarity_on_tuples([("slim", "woman"), ("slim", "man"), ("slimness", "woman"), ("slimness", "man"), ("slender", "woman"), ("slender", "man")], second_period_model)

Cosine Similarity between 'slim' and 'woman' is 0.5575359463691711
Cosine Similarity between 'slim' and 'man' is 0.3135552704334259
Cosine Similarity between 'slimness' and 'woman' is 0.517393171787262
Cosine Similarity between 'slimness' and 'man' is 0.3480556309223175
Cosine Similarity between 'slender' and 'woman' is 0.5198826789855957
Cosine Similarity between 'slender' and 'man' is 0.402288556098938


[('slim', 'woman', np.float32(0.55753595)),
 ('slim', 'man', np.float32(0.31355527)),
 ('slimness', 'woman', np.float32(0.5173932)),
 ('slimness', 'man', np.float32(0.34805563)),
 ('slender', 'woman', np.float32(0.5198827)),
 ('slender', 'man', np.float32(0.40228856))]

Un ulteriore argomento per il quale un'analisi diacronica mi sembra sensata è quello della professione della scrittrice e dei generi letterari a essa maggiormente legati, dove si nota ancora un assottigliamento nelle differenze delle *cosine similarity* calcolate. Tali divari vanno misurati tra le parole di ciascuna coppia (es: *writer*-*man*; *writer*-*woman*).

In [ ]:
compute_similarity_on_tuples([("writer", "woman"), ("writer", "man"), ("poet", "man"), ("poet", "woman"), ("novelist", "woman"), ("novelist", "man")], first_period_model)

Cosine Similarity between 'writer' and 'woman' is 0.4905639588832855
Cosine Similarity between 'writer' and 'man' is 0.6125530004501343
Cosine Similarity between 'poet' and 'man' is 0.5203648805618286
Cosine Similarity between 'poet' and 'woman' is 0.45095688104629517
Cosine Similarity between 'novelist' and 'woman' is 0.5175894498825073
Cosine Similarity between 'novelist' and 'man' is 0.42237338423728943


[('writer', 'woman', np.float32(0.49056396)),
 ('writer', 'man', np.float32(0.612553)),
 ('poet', 'man', np.float32(0.5203649)),
 ('poet', 'woman', np.float32(0.45095688)),
 ('novelist', 'woman', np.float32(0.51758945)),
 ('novelist', 'man', np.float32(0.42237338))]

In [ ]:
compute_similarity_on_tuples([("writer", "woman"), ("writer", "man"), ("poet", "man"), ("poet", "woman"), ("novelist", "woman"), ("novelist", "man")], second_period_model)

Cosine Similarity between 'writer' and 'woman' is 0.41301053762435913
Cosine Similarity between 'writer' and 'man' is 0.5475563406944275
Cosine Similarity between 'poet' and 'man' is 0.4084462821483612
Cosine Similarity between 'poet' and 'woman' is 0.4518347680568695
Cosine Similarity between 'novelist' and 'woman' is 0.44298213720321655
Cosine Similarity between 'novelist' and 'man' is 0.43773728609085083


[('writer', 'woman', np.float32(0.41301054)),
 ('writer', 'man', np.float32(0.54755634)),
 ('poet', 'man', np.float32(0.40844628)),
 ('poet', 'woman', np.float32(0.45183477)),
 ('novelist', 'woman', np.float32(0.44298214)),
 ('novelist', 'man', np.float32(0.4377373))]

---

Prima di terminare la ricerca di differenze nelle associazioni semantiche tra i due periodi, ho voluto aggiungere delle analogie sempre relative alla scrittura, ma in senso lato, intesa come atto creativo.

Nell'analogia *imagination*-*woman*-*man* si può notare come le parole "incognite" vengono rappresentate da termini neutri e più affini all'accezione di "immaginazione" come azione, facoltà o abilità della mente (*intellection*, *mind*, *conception*, *ideation*). Al contrario, nell'analogia *imagination*-*man*-*woman* appaiono già parole appartenenti al campo semantico della magia, del fascino e, pertanto, dell'irrazionalità.

Questi sono i risultati per il primo periodo:

In [ ]:
first_period_model.get_analogies("imagination", "woman", "man", k = 20)

[(0.7761602997779846, 'imaginations'),
 (0.7737412452697754, 'conceptions'),
 (0.7648842930793762, 'immagination'),
 (0.7063747644424438, 'conceptive'),
 (0.7037827968597412, 'intellection'),
 (0.7036216259002686, 'imaginative'),
 (0.6990354061126709, 'realities'),
 (0.697597861289978, 'mind'),
 (0.6969273090362549, 'intellectually'),
 (0.6932141184806824, 'ideation'),
 (0.6914507746696472, 'creations'),
 (0.6895903944969177, 'perceptions'),
 (0.6868717670440674, 'conception'),
 (0.6748239994049072, 'imaginatively'),
 (0.6738391518592834, 'perception'),
 (0.6703128218650818, 'preconceptions'),
 (0.6687296628952026, 'inception'),
 (0.6686314940452576, 'reflection'),
 (0.6643115282058716, 'intuitions'),
 (0.6577197313308716, 'intuition')]

In [ ]:
first_period_model.get_analogies("imagination", "man", "woman", k = 20)

[(0.7751824855804443, 'fancy'),
 (0.680626392364502, 'ecstacy'),
 (0.6703117489814758, 'imaginations'),
 (0.6643052101135254, 'imagery'),
 (0.657641589641571, 'imaginative'),
 (0.6515043377876282, 'womanish'),
 (0.6490534543991089, 'novelty'),
 (0.6390591859817505, 'enraptures'),
 (0.6379117965698242, 'immagination'),
 (0.6371483206748962, 'charms'),
 (0.6342476606369019, 'illusion'),
 (0.6317763328552246, 'enrapture'),
 (0.6278595328330994, 'voluptuousness'),
 (0.6278356909751892, 'bewitching'),
 (0.6274372339248657, 'bewitch'),
 (0.6253401637077332, 'bewitches'),
 (0.6240952610969543, 'fascinations'),
 (0.623597264289856, 'fascination'),
 (0.6235331892967224, 'imaginatively'),
 (0.6226822137832642, 'loveliness')]

Questi, invece, sono i risultati ottenuti per il secondo periodo. Al contrario di quanto accaduto sopra, qui i risultati non hanno permesso l'identificazione di un cambio semantico, essendo ancora presenti parole affini come *fascinate*, *witchery* e *daydreams* da una parte e termini più denotativi dall'altra.

In [ ]:
second_period_model.get_analogies("imagination", "man", "woman", k = 20)

[(0.7566232085227966, 'imaginings'),
 (0.7531231641769409, 'imaginative'),
 (0.7377762198448181, 'imaginations'),
 (0.7301519513130188, 'imaginativeness'),
 (0.6845893859863281, 'imagining'),
 (0.6807485222816467, 'unimaginative'),
 (0.6793488264083862, 'fascinate'),
 (0.6789688467979431, 'romantics'),
 (0.6788833737373352, 'fancy'),
 (0.6725452542304993, 'imaginatively'),
 (0.6720461845397949, 'witchery'),
 (0.6704432368278503, 'romantic'),
 (0.6550822257995605, 'daydreams'),
 (0.6538625359535217, 'unreality'),
 (0.651884138584137, 'womanʼs'),
 (0.648598313331604, 'womanhood'),
 (0.6475205421447754, 'dreams'),
 (0.6450551748275757, 'phantasmagoria'),
 (0.6431597471237183, 'girlhood'),
 (0.6429632902145386, 'loveliness')]

In [ ]:
second_period_model.get_analogies("imagination", "woman", "man", k = 20)

[(0.7289305329322815, 'imaginations'),
 (0.7081634402275085, 'pagination'),
 (0.6426495909690857, 'creations'),
 (0.6333258152008057, 'conception'),
 (0.6310347318649292, 'imaginative'),
 (0.6245633363723755, 'imaginatively'),
 (0.6236556172370911, 'intellection'),
 (0.606531023979187, 'inspirations'),
 (0.6065067648887634, 'imaginativeness'),
 (0.6060525178909302, 'conceptions'),
 (0.6043369770050049, 'interestingness'),
 (0.6016435027122498, 'real'),
 (0.6003077030181885, 'stimulations'),
 (0.5995008945465088, 'fancies'),
 (0.5983129143714905, 'realises'),
 (0.5977376699447632, 'mind'),
 (0.5960102081298828, 'origination'),
 (0.595865786075592, 'ideation'),
 (0.5922331213951111, 'stimulation'),
 (0.5905252695083618, 'brain')]

La stessa ricerca tramite analogie è stata effettuata sostituendo a *imagination* la parola *intellect*, anche questa volta senza trovare segnali di un avvenuto cambio semantico. Sempre ricalcando i risultati precedenti, tuttavia, è ancora più evidente il divario nella considerazione dell'attività intellettuale, creativa e non, negli uomini e nelle donne:

In [ ]:
first_period_model.get_analogies("intellect", "woman", "man", k = 20)

[(0.8133224844932556, 'intellectually'),
 (0.7720144391059875, 'intellection'),
 (0.7690684199333191, 'intellectual'),
 (0.7670629024505615, 'intellectuality'),
 (0.752723753452301, 'intellects'),
 (0.7379862070083618, 'faculties'),
 (0.7122082114219666, 'understanding'),
 (0.7110635042190552, 'intellectuals'),
 (0.6975119113922119, 'mind'),
 (0.674544095993042, 'conceptions'),
 (0.6723259091377258, 'mental'),
 (0.6697984933853149, 'capacities'),
 (0.6625398993492126, 'capabilities'),
 (0.6601700782775879, 'profoundnesses'),
 (0.6589595079421997, 'faculty'),
 (0.6577504277229309, 'understandings'),
 (0.6576583385467529, 'unintellectual'),
 (0.6552495360374451, 'conceptive'),
 (0.652830958366394, 'rationality'),
 (0.6517380475997925, 'profounder')]

In [ ]:
first_period_model.get_analogies("intellect", "man", "woman", k = 20)

[(0.6913699507713318, 'intellects'),
 (0.6630282402038574, 'unwomanly'),
 (0.6574728488922119, 'comprehensibility'),
 (0.6524357199668884, 'girl'),
 (0.652434766292572, 'unconsciousness'),
 (0.6478558778762817, 'intellectuals'),
 (0.6474421620368958, 'loveliness'),
 (0.6468258500099182, 'incomprehensibilities'),
 (0.6466469168663025, 'womanish'),
 (0.6391618847846985, 'intellectual'),
 (0.6386112570762634, 'unendowed'),
 (0.6365519762039185, 'sensibilities'),
 (0.6363183856010437, 'incomprehensibility'),
 (0.6328036785125732, 'unintellectual'),
 (0.6323248744010925, 'idiotcy'),
 (0.631449818611145, 'sensibility'),
 (0.6244667768478394, 'infantile'),
 (0.621767520904541, 'insensibility'),
 (0.6163738369941711, 'susceptibility'),
 (0.6159411072731018, 'susceptibilities')]

Rimanendo nell'ambito della produzione letteraria, in entrambi i casi la spiegazione si può rintracciare in vari esempi che Sandra Gilbert e Susan Gubar hanno inserito nella loro già brevemente citata opera di critica letteraria femminista *The Madwoman in the Attic: The Woman Writer and the Nineteenth-Century Literary Imagination*: le attività mentali, tra cui quella immaginativa, venivano considerate generalmente come pericolose per la salute mentale delle donne (72). Le stesse autrici coltivavano un complesso di inferiorità che era il contrappeso di un senso comune che definiva come superiori i risultati ottenuti dagli uomini nell'uso della loro intelligenza creativa: nel diciassettesimo e diciottesimo secolo (fino al diciannovesimo, in alcuni casi) non era raro che le scrittrici si mostrassero perlomeno tenute a giustificare un comportamento tanto arrogante come lo era per loro dedicarsi alla scrittura; se rifiutavano di prendere parte a questa tradizione di autocritica diventavano oggetto di attacchi alla propria opera e persona, e/o di accuse di plagio (Gilbert e Gubar 76-79).  

## *Word Embeddings* in ambito DH: una breve panoramica di studi affini elaborati in ambito accademico

L’individuazione di possibili cambi semantici tramite analisi e strumenti computazionali può avere applicazioni che non si limitano al rintracciamento di tendenze macroscopiche nel discorso pubblico a partire da corpora di testi prodotti in determinati periodi storici: un uso "ancillare" dei *Word Embeddings* può essere quello di aiutare a evitare considerazioni sbagliate sul valore semantico delle parole analizzate, il quale può certamente cambiare da un’epoca all’altra; un esempio interessante è lo studio di Lettieri et al., in cui si analizzano differenze osservabili in testi letterari scritti da uomini e donne con l’intento di quantificare la ricorrenza di termini appartenenti a determinate categorie grammaticali e semantiche (2022, 16). Più precisamente, in questa sezione ci si concentrerà sulla descrizione di studi elaborati in ambito accademico e che si avvicinano al tipo di progetto presentato in questo notebook. Pertanto, gli stessi sono stati selezionati in base alla loro vicinanza in termini di obiettivo di ricerca, di strumenti e di metodi utilizzati. In nessuno di essi si usa fastText, il che, pur dando spunti di riflessione sull'opportunità di usare il metodo a *subwords* rispetto a quello classico di word2vec, non è stato visto come un problema: oltre alla possibilità aggiuntiva che la libreria di Facebook offre di gestire anche quelle parole non presenti nel vocabolario, in generale essa è descritta come "un'estensione" dello stesso word2vec (v. esempio in Jurafsky e Martin 2025, 16).      

Nel primo studio preso in considerazione, Verheul et al. (2022), l’oggetto d'indagine è il cambio semantico delle parole usate per esprimere determinati “concetti”, o “astrazioni o rappresentazioni mentali che hanno funto da mattoni per la costruzione di pensieri e credenze” nel corso del periodo analizzato (4); analogamente a quanto fatto in questo progetto, anche nello studio si relazionano i risultati delle *cosine similarity* con fenomeni di carattere sociale e storico. Sono stati usati testi appartenenti a tipologie diverse e l'algoritmo di word2vec (con parametri però simili). Si analizza un periodo di tempo più breve ma frammentato in micro-intervalli di dieci anni, generando pertanto una visione del cambio semantico più graduale. La portata dei corpora è volutamente transnazionale ma ben compartimentata in contesti sociali e geografici distinti per ciascuno di essi, caratteristica non condivisa con i corpora da me usati, creati con dei procedimenti che hanno limitato il controllo che avrei potuto avere sulla scelta della nazionalità degli autori e sul luogo di pubblicazione dei documenti.

Mentre nello studio di Verheul et al. si analizzano esclusivamente i “vicini” semantici delle parole target, in Sökefeld e Amaral (2025) anche le analogie sono fonti di informazioni rilevanti per quel che riguarda gli stereotipi di genere impliciti nei documenti utilizzati per analizzare i modelli. Oltre all’identificazione della sessualizzazione a cui sono sottoposte nel tempo parole che si riferiscono specificamente a figure femminili, le autrici esaminano i risultati di analogie relative a ruoli familiari e a posizioni di potere scoprendo che anche in casi dove il quarto elemento apparirebbe scontato (“principe sta a principessa come x sta ad arciduca”) in realtà è la categoria semantica di riferimento dell’intera analogia a prevalere sul genere della parola che dovrebbe rappresentare il risultato stesso: in riferimento al campo semantico del potere, infatti, la maggior parte dei termini risultanti sono maschili; viceversa nelle analogie riguardanti parole appartenenti al campo semantico delle relazioni familiari (Sökefeld e Amaral 2025, 8). Tale analisi può essere messa sullo stesso piano delle osservazioni fatte in questo notebook sull’accostamento di termini relativi alla sessualità e alla nevrosi, sulla presenza di sostantivi riguardanti professioni di basso prestigio sociale o confinate in un ambito domestico nell’analogia “master-man-woman”, o ancora sulla scarsità di termini specificamente femminili nell’analogia “workman-man-woman”.

Altrettanto rilevante e ancora più vicino allo scopo del progetto presentato in questo notebook è lo studio di Pedrazzini e McGillivray (2022), nel quale ci si propone di paragonare i risultati ottenuti da modelli di *WE* allenati su un corpus di grandi dimensioni di articoli di giornale britannici a studi qualitativi e di *close reading* sul cambio semantico occorso per alcune parole durante l’epoca vittoriana. Gli stessi autori sottolineano i vantaggi dell’uso di pubblicazioni periodiche per l’allenamento di questo tipo di modelli, essendo più semplici da filtrare secondo criteri quali il luogo e la data di pubblicazione, e l’ideologia politica di riferimento. Non è un caso, quindi, che molti degli studi incontrati nella ricerca di fonti di ispirazione e/o riferimenti dello stato dell’arte per il mio progetto (tra cui quello già descritto di Verheul et al.) abbiano molto spesso questa caratteristica in comune.

Inoltre, lo studio di Pedrazzini e McGillivray offre un ulteriore esempio di passaggio effettuato in molti dei più recenti studi da me letti: l’allineamento degli spazi vettoriali di ciascun periodo in modo tale da misurare automaticamente l’avvenimento e l’entità dello stesso cambio semantico nel tempo. I modelli di *WE*, difatti, producono vettori diversi per le stesse parole anche se il corpus su cui si allenano è il medesimo; risulterebbe sconveniente calcolare la *cosine similarity* dei vettori prodotti per una parola da due modelli allenati in turni diversi (Kutuzov et al. 2018, 1389). Nel notebook, in assenza di questa procedura di allineamento, la presenza, oltre che l’importanza stessa, del cambio semantico è stata individuata e misurata tenendo in considerazione le distanze delle *cosine similarities* relative ai singoli periodi, separando i risultati dati dai due modelli.  

### Conclusioni

Questi studi sono stati selezionati con l’intento di mostrare degli esempi di usi analoghi di *embeddings* nella detezione e analisi del cambio semantico diacronico. Come si può notare, in vari l’approccio è, almeno in parte, pressochè identico a quello adottato nel presente progetto; ovviamente, lo stesso si può dire per quelle ricerche in cui gli strumenti (e soprattutto le conoscenze) a disposizione sono più avanzati di quelli in mia dotazione. Ciononostante, pur essendosi trattato di un primo contatto con la potenzialità degli *Word Embeddings* di catturare i significati delle parole, il semplice uso che di fastText è stato fatto qui ha dato risultati già soddisfacenti nel rilevamento (verificabile tramite fonti in cui il metodo adottato è stato qualitativo e di *close reading*) di certe sfumature di significato attribuite a certe parole nel corso dei periodi presi in considerazione. In aggiunta, nonostante alcuni aspetti della qualità dei dati a disposizione (non lemmatizzati né selezionati in base a criteri come la frequenza minima o normalizzati nelle possibili varianti ortografiche in essi contenute), si è visto come la quantità degli stessi sia stata sufficiente a far affiorare affinità semantiche utili allo svolgimento dell'analisi. Quest'ultimo punto rivela una caratteristica positiva di fastText e ben si armonizza con la sempre maggiore disponibilità di contenuto testuale nel Web. Tuttavia, a mio avviso è importante ricordare che talvolta i criteri di costruzione dei corpora possono essere assai più restrittivi e rendere più complicata la fase di reperimento automatico dei testi (e.g. lingua diversa dall'inglese e con poca rappresentazione, testi pubblicati in una determinata area geografica o appartenenti alla produzione di un autore specifico; senza contare la difficoltà nel trovare materiale di qualità non protetto da diritti d'autore o disponibile gratuitamente). Nel caso di eventuali progetti futuri potrebbe essere interessante partire da corpora più ristretti, modificare le scelte di *preprocessing* effettuate nel primo notebook e valutare i risultati ottenuti con parametri diversi per i modelli, o addirittura confrontare fastText con l'algoritmo di word2vec. In modo tale da ottenere manifestazioni più graduali del cambio semantico si potrebbero infine suddividere i corpora in intervalli di tempo più brevi, in decadi piuttosto che in secoli.

Un'ultima precisazione riguarda l'uso di ChatGPT e altri strumenti di IA come Gemini in Google Colab, dove è stato sviluppato il primo notebook. Le parti in cui sono stati adoperati hanno riguardato solo il codice Python e compiti per i quali avevo già avuto prove, in altri progetti, di un funzionamento efficace: la creazione di funzioni in cui è previsto l'uso di RegEx, la gestione di errori riguardanti librerie e procedimenti nuovi o da me poco usati, oltre che nella ricerca di soluzioni rapide in situazioni di stallo, come fatto per l'attribuzione di colori diversi a parole appartenenti a *cluster* di "vicini" diversi dopo l'applicazione della PCA (v. funzione **visualise_neighbours**). L'intenzione è sempre stata quella di capire i risultati dati dalla IA attraverso controlli e riformulazioni personali del codice suggerito e, nel caso in cui non ci fossi riuscita, di non integrarlo nel progetto.  


## Bibliografia

Bailey, Joanne. 2002. «Favoured or oppressed? Married women, property and ‘coverture’ in England, 1660–1800». *Continuity and Change* 17 (3): 351–72. https://doi.org/10.1017/S0268416002004253.

Begiato, Joanne. 2023. «A “Master-Mistress”: Revisiting the History of Eighteenth-Century Wives.» *Women’s History Review* 32 (1): 1–20. Humanities Source Ultimate (161545532). https://doi.org/10.1080/09612025.2022.2028362.

Brontë, Charlotte. 1999. *Jane Eyre*. Wordsworth Classics. Wordsworth Editions.

Firth, J. 1957. «A Synopsis of Linguistic Theory, 1930-1955». https://www.semanticscholar.org/paper/A-Synopsis-of-Linguistic-Theory%2C-1930-1955-Firth/88b3959b6f5333e5358eac43970a5fa29b54642c.

Franklin, Caroline. 2003. «Feminism 1890-1940». In *Encyclopedia of Literary Modernism*, a cura di Paul Poplawski. Greenwood.

Frawley, Maria. 2017. «The Victorian Age, 1832-1901». In *English Literature in Context*, 2a. CUP.

Gilbert, Sandra M., e Susan Gubar. 2020. *The Madwoman in the Attic: The Woman Writer and the Nineteenth-Century Literary Imagination*. Yale University Press.

Harris, Zellig S. 1954. «Distributional Structure». *WORD*, agosto 1. world. https://www.tandfonline.com/doi/abs/10.1080/00437956.1954.11659520.

Joos, Martin. 1950. «Description of Language Design». *The Journal of the Acoustical Society of America* 22 (6): 701–7. https://doi.org/10.1121/1.1906674.

Jurafsky, Daniel, e James H. Martin. 2025. «Embeddings». In *Speech and Language Processing: An Introduction to Natural Language Processing, Computational Linguistics, and Speech Recognition with Language Models*, 3rd ed. https://web.stanford.edu/~jurafsky/slp3/.

Kitson, Peter J. 2017. «The Romantic Period, 1780-1832». In *English Literature in Context*, 2a. CUP.

Kutuzov, Andrey, Lilja Øvrelid, Terrence Szymanski, e Erik Velldal. 2018. «Diachronic word embeddings and semantic shifts: a survey». In *Proceedings of the 27th International Conference on Computational Linguistics*, a cura di Emily M. Bender, Leon Derczynski, e Pierre Isabelle. Association for Computational Linguistics. https://aclanthology.org/C18-1117/.

Lettieri, Giada, Giacomo Handjaras, Erika Bucci, Pietro Pietrini, e Luca Cecchetti. 2022. «Sex Differences in Literature Have Diminished over Time and Are Less Pronounced in More Developed Countries». Preprint, PsyArXiv. https://doi.org/10.31234/osf.io/wa2xf.

Levy, Omer, e Yoav Goldberg. 2014. «Dependency-Based Word Embeddings». *Proceedings of the 52nd Annual Meeting of the Association for Computational Linguistics (Volume 2: Short Papers)*, 302–8. https://doi.org/10.3115/v1/P14-2050.

Oxford English Dictionary. 2024a. *feminist, adj. & n.* Oxford University Press, marzo. Oxford English Dictionary. https://doi.org/10.1093/OED/9678491930.

Oxford English Dictionary. 2024b. *suffragist, n. & adj.* Oxford University Press, dicembre. Oxford English Dictionary. https://doi.org/10.1093/OED/2049527380.

«Oxford English Dictionary». s.d. Consultato 27 giugno 2025. https://www.oed.com/.

Pedrazzini, Nilo, e Barbara McGillivray. 2022. «Machines in the media: semantic change in the lexicon of mechanization in 19th-century British newspapers». In *Proceedings of the 2nd International Workshop on Natural Language Processing for Digital Humanities*, a cura di Mika Hämäläinen, Khalid Alnajjar, Niko Partanen, e Jack Rueter. Association for Computational Linguistics. https://doi.org/10.18653/v1/2022.nlp4dh-1.12.

Showalter, Elaine. 1987. *The Female Malady: Women, Madness and English Culture*, 1830-1980. Penguin Books.

Showalter, Elaine. 1992. «Introduction». In *Mrs. Dalloway*. Penguin Books.

Shuttleworth, Sally. 1997. *Charlotte Brontë and Victorian Psychology*. CUP.

Sökefeld, Carla, e Patrícia Amaral. 2025. «Semantic Change of Female-Denoting Nouns in Diachronic German Corpora». *Proceedings of the Linguistic Society of America* 10 (1): 1. https://doi.org/10.3765/plsa.v10i1.5905.

Stone, Lawrence. 1983. *Famiglia, sesso e matrimonio in Inghilterra tra Cinque e Ottocento*. Biblioteca di cultura storica 150. Einaudi.

Verheul, Jaap, Hannu Salmi, Martin Riedl, et al. 2022. «Using word vector models to trace conceptual change over time and space in historical newspapers, 1840–1914». *Digital Humanities Quarterly* 016 (2).